<a href="https://colab.research.google.com/github/ebatty/IntroCompNeuro/blob/main/lectures/20_HebbianLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 20 - Hebbian & Unsupervised Learning

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Section 1: Hebbian Learning Rules

In 1949, David Hebb postulated that "neurons that fire together, wire together".  In other words, if a presynaptic neuron and postsynaptic neuron are firing at the same time, the synapse between them strengthens. 

## Section 1.1: Experimental evidence of Hebbian learning: long-term potentiation (LTP)


<img src="https://raw.githubusercontent.com/ebatty/IntroCompNeuro/main/images/LTP.png" width = "600">


There is some experimental evidence of this in the brain (schematic above from https://neuronaldynamics.epfl.ch/online/Ch19.S1.html). We can record the excitatory postsynaptic potential (EPSP) in a postsynaptic neuron - this is the voltage change in the postynaptic cell body following a presynaptic spike. We can stimulate the presynaptic fibers a little - enough to induce a postynaptic voltage change but not enough to induce a postsynaptic spike (Part A in figure above). We measure the strength of the postsynaptic response (the EPSP amplitude). We can then stimulate the presynaptic fibers a lot - enough to induce postsynaptic spikes so the presynaptic and postsynaptic neurons are firing together (Part B above). After this, we test the subthreshold presynaptic stimulation and measure the EPSP amplitude (part C above). We find that the EPSP amplitude is bigger after the period where the presynaptic and postsynaptic neurons were firing together - meaning that the neurons wired together because they fired together (part D above)! This effect is called **long-term potentiation** and can last for days or months. 

We also see **long-term depression** experimentally: if a presynaptic neuron is firing for an extended period of time and the postsynaptic neuron never fires, the synapse can weaken.

## Section 1.2: Our neural model

<img src="https://raw.githubusercontent.com/ebatty/IntroCompNeuro/main/images/neuraldiagram.png" width = "300">

Assume we have one output neuron receiving inputs from several input neurons (in the schematic above, 2). We will use the same notation as for our RNNs: we have an output neuron with firing rate $v$ receiving synaptic connections from some inputs neurons, whose firing rates are summarized in vector $\bar{u} = \begin{bmatrix}u_1 \\ u_2 \\ \end{bmatrix}$. So the first element of $\bar{u}$ is the first input neuron firing rate and the second element is the second input neuron firing rate (and so on if we had more input neurons). The weights from the input neurons to the output neuron are summarized in vector $\bar{w} = \begin{bmatrix}w_1 \\ w_2 \\ \end{bmatrix}$, where the first element is the weight from the first input neuron, the second element is the weight from the second input neuron (and so on if we had more input neurons). 

Our neural model is that the output neural firing rate is the weighted sum of the inputs, where the weights are...the synaptic weights! Specifically, our output neuron firing rate is the dot product between $\bar{w}$ and $\bar{u}$:

\begin{align}
v = \bar{w} \cdot \bar{u}
\end{align}

## Section 1.3: Basic Hebbian Learning

We can formalize the idea that neurons that fire together, wire together, with the following equation:

\begin{align}
\tau_w \frac{d\bar{w}}{dt} = v(t) \bar{u}(t)
\end{align}

In other words, the rate of change in the weight between input neuron and output neuron is equal to the output firing rate times the input firing rate divided by the time constant $\tau_w$.  Remember that the output and input neuron firing rates change over time. So the weight will only grow if both neurons are active simultaneously (the whole idea of Hebbian learning).

Below, I simulate Hebbian learning in code. I plot $v$ and $u_1$ over time. I also plot both $\frac{w_1}{dt}$ and $w_1$ over time, computed with Hebbian learning. You can see that the weight grows when the output neuron and the first input neuron are active at the same time. 

In [ ]:
T = 200 # number of time steps to simulate for
tau = 1
dt = 0.001

# Create firing rates of two input neurons
u = np.zeros((T, 2))
u[50:100, 0] = 5
u[110:190, 1] = 1

# Initialize weights (keep track for plotting)
w = np.zeros((T, 2))
w[0] = .1*np.ones((2, ))

# Initialize dw_dt and v (just tracking for plotting)
dw_dt = np.zeros((T, 2))
v = np.zeros((T, ))

# Loop over time steps
for i_t in range(T - 1):

  # Compute output firing rate according to model
  v[i_t] = np.dot(w[i_t], u[i_t])

  # Compute dw_dt
  dw_dt[i_t] = v[i_t] * u[i_t] / tau

  # Update w
  w[i_t + 1] = w[i_t] + dw_dt[i_t] * dt

fig, axes = plt.subplots(4, 1, figsize = (10, 5), sharex = True)

axes[0].plot(u)
axes[0].legend([r'Input neuron 1: $u_1$', r'Input neuron 2: $u_2$'])

axes[1].plot(v, 'g')
axes[1].legend([r'Output neuron: v'])

axes[2].plot(dw_dt)
axes[2].legend([r'$\frac{dw_1}{dt}$', r'$\frac{dw_1}{dt}$'])

axes[3].plot(w)
axes[3].legend([r'$w_1$', r'$w_2$'])

With this Hebbian learning rule is that our weights can only increase, not decrease: both $v$ and $\bar{u}$ represent firing rates and so are non-negative, meaning that their product and by extension $\frac{dw}{dt}$ is always positive. There are two issues: 1) the weights would evenutally explore, and 2) this doesn't model long-term depression. We'll fix the second issue first in the next part.



## Section 1.4: Covariance-based learning

To model long-term depression, we extend our learning rule:

\begin{align}
\tau_w \frac{d\bar{w}}{dt} = \bar{u}(t)(v(t) - <v>)
\end{align}

We are now modeling the rate of change in the weight as the input neuron firing rate times the output neuron firing rate at that time minus its average firing rate, $<v>$, over the time constant. The average firing rate $<v>$ is the average firing rate of the output neuron over some prior chunk of time. 

If the output firing rate is higher than average, the term $(v(t) - <v>)$ will be positive and the synapses will strengthen, modeling long-term potentiation. If the output firing rate is lower than average, the term $(v(t) - <v>)$ is negative, and the synapses will weaken, consistent with long-term depression. This makes intuitive sense: we don't just care whether the output neuron is firing at the same time as the input neuron, we care whether it's *firing more than it usually does*.


So, we've solved our problem from our first rule of not allowing synapses to weaken - this extension models LTP in addition to LTD! Unfortunately, despite that, with this model, the weights still tend to explode to infinity. See the next section for proof.


This learning rule is called **covariance-based learning**. We can average over the training inputs: in other words we assume that the weights are the same while the neuron sees that set of inputs and just change afterwards. This is a fair approximation because the time scale of learning is often much slower than the time scale of seeing multiple inputs. If we do that, we can write:

\begin{align}
\tau_w \frac{d\bar{w}}{dt} &= <\bar{u}(t)(v(t) - <v>)> \\
&= <\bar{u}(\bar{u}\cdot \bar{w} - <\bar{u}\cdot \bar{w}>)>\\
&= <\bar{u}(\bar{u} - <\bar{u} >)> \cdot \bar{w}
\end{align}

It so happens that this first term, $<\bar{u}(\bar{u} - <\bar{u} >)>$, defines the covariance matrix of the inputs, $C$. If we had N input neurons, the covariance matrix is an N x N matrix where each entry denotes how those two input neurons (indicated by the row and column) covary. This means we can write this rule as:

\begin{align}
\tau_w \frac{d\bar{w}}{dt} &= C \cdot \bar{w}
\end{align}

In other words, the rate of change of the weights equal the covariance matrix of the inputs over a period of time times the starting weight vector. 

## Section 1.5: Proof of weight explosion

**Basic Hebb's Rule** (Section 1.3)

Let's return to our basic Hebbian rule and prove that the weights will explode. Remember our neural model is:

\begin{align}
v = \bar{w} \cdot \bar{u}
\end{align}

and our learning rule is

\begin{align}
\tau \frac{d\bar{w}}{dt} = v\bar{u}
\end{align}

We can write the square of the length of the weight vector, $||\bar{w}||^2$ as the dot product of the weight vector with itself (this is relation from linear algebra that the length of a vector equals the square root of its dot product with itself).

\begin{align}
||\bar{w}||^2 = \bar{w}\cdot\bar{w}
\end{align}

Let's compute the derivative of the length of this vector squared. The first step is to use the chain rule:

\begin{align}
\frac{d||\bar{w}||^2}{dt} &= 2\bar{w}\cdot\frac{d\bar{w}}{dt}\\
&= 2\bar{w}\cdot(v\bar{u})/\tau\\
&= 2v\bar{w}\cdot\bar{u}/\tau\\
&= 2v^2/\tau
\end{align}

Between the first and second rows, we substituted in the equation for $\frac{d\bar{w}}{dt}$. Between the third and fourth rows, we substituted in $v = \bar{w}\cdot\bar{u}$. We see that the derivative of the length of the weight vector squared is equal to 2 times the output firing rate squared divided by the time constant. These terms can never be negative, or equal to zero, so the length of the weight vector will grow and grow until it explodes!

**Covariance-based rule** (Section 1.4)

Even though the covariance-based rule presented in section 1.4 allows for long-term depression, the weights still explode. We can prove this in much the same way as above, by deriving the derivative of the length of the weight vector squared, but the proof is a little more difficult/confusing so we'll skip it (but check Dayan & Abbott Chapter 8 if interested).


## Section 1.6: Oja's rule

We will look at one more extension of the Hebbian learning rule: Oja's rule. This rule results in more stable learning: the synaptic weights do not explode. Oja's rule is:

\begin{align}
\tau \frac{d\bar{w}}{dt} = v\bar{u} - \alpha v^2\bar{w}
\end{align}

where $\alpha$ is some constant value.

As in section 1.5, we can compute the derivative of the length of the weight vector squared:


\begin{align}
\frac{d||\bar{w}||^2}{dt} &= 2\bar{w}\cdot\frac{d\bar{w}}{dt}\\
&= 2\bar{w}\cdot(v\bar{u} - \alpha v^2\bar{w})/\tau\\
&= (2v\bar{w}\cdot\bar{u} - 2\alpha v^2\bar{w}\cdot\bar{w})/\tau\\
&= 2v^2 - 2\alpha v^2||\bar{w}||^2\\
&= 2v^2(1 - \alpha ||\bar{w}||^2)
\end{align}

We can find the steady-state values of the weights by setting this equal to zero and solving for the length of the weight vector. We find that the length of the weight vectors becomes the square root of 1 over $\alpha$:

\begin{align}
||\bar{w}|| = \sqrt{\frac{1}{\alpha}}
\end{align}

The weights do not explode as the length of the weight vector does not explode!

Oja's rule induces **competition between synapses**: if one synapse strengthens, the others must weaken so that the length of the total vector remains at $\sqrt{\frac{1}{\alpha}}$.


# Section 2: What does Hebbian learning learn?

 

## Section 2.1: Simulation

So what does Hebbian learning actually learn? Let's look at an example where our two input neurons are correlated. In the plot below, each point is the firing rate of the two neurons at a specific time.

In [ ]:
# Create inputs
T = 300
u1 = np.random.randn(T)
u2 = 2*u1 + .5*np.random.randn(T,)
u = np.concatenate((u1[:, None], u2[:, None]), axis = 1)

plt.plot(u[:, 0], u[:, 1], '.k')
plt.xlabel(r'$u_1$')
plt.ylabel(r'$u_2$')

We can implement Oja's rule and update the weights for each data point, as below.

In [ ]:
tau = .1
dt = .1
alpha = 1

w = .1 * np.ones((2, ))
v = np.zeros((T, 1))

for step in range(T):

  # Compute output response
  v[step] = np.dot(w, u[step])

  # Compute rate of change of weights
  dw_dt = v[step] * u[step]  - alpha * v[step]**2 * w

  # Update weights
  w += dw_dt * dt

In [ ]:
plt.plot(u[:, 0], u[:, 1], '.k')
plt.plot([0, w[0]], [0, w[1]], '-r', linewidth = 4)
plt.plot(w[0], w[1], '-or', linewidth = 4, label = 'Learned weights')
plt.legend()
plt.xlabel(r'$u_1$')
plt.ylabel(r'$u_2$')

The red line is the weight vector. You may be able to tell by eye that the weight vector learned is the first principal component of the input data!

## Section 2.2: Mathematical proof

We can prove what we see in our simulation: Hebbian learning results in a weight vector that equals the first principal component. Let's prove this for our covariance-based rule. 

Remember our rule is:

\begin{align}
\tau_w \frac{d\bar{w}}{dt} = \bar{u}(t)(v(t) - <v>)
\end{align}

We can also write it as

\begin{align}
\tau_w \frac{d\bar{w}}{dt} = C \cdot \bar{w}
\end{align}

where C is the covariance matrix.

Let's say we have the eigenvectors of the covariance matrix $\bar{e}_1$, $\bar{e}_2$, and so on, and the corresponding eigenvalues $\lambda_1$, $\lambda_2$, and so on. Covariance matrices are symmetric, so just as with the recurrent matrix in RNNs, we know that the eigvectors form a basis for N-dimensional space. This means that we can write any vector in that space as a linear combination of the eigenvectors. So any weight vector can be written as the sum of scalar values, c's, times the eigenvectors:

\begin{align}
\bar{w}(t) = \sum_i c_i(t)\bar{e}_i
\end{align}

We can find the equation for each coefficient (not shown as similar to RNN case), which is

\begin{align}
c_i(t) = c_i(0)exp(\frac{\lambda_i t}{\tau})
\end{align}

So, we get 

\begin{align}
\bar{w}(t) = \sum_i c_i(0)exp(\frac{\lambda_i t}{\tau})\bar{e}_i
\end{align}

Let's say we have two eigenvalues, $\lambda_1 = 0.9$, and $\lambda_2 = 0.6$. Below, I plot $exp(\frac{\lambda_1 t}{\tau})$ in green and $exp(\frac{\lambda_2 t}{\tau})$ in red, with $\tau = 1$ for t up to 1000. You can see that at high t, the term with the larger lambda is much much bigger: the red line looks like it stays at zero by comparison (even though it actually reaches up to 2 x 10^260. At large t, the term with the largest eigenvalue dominates in the sum, so much so that we can ignore the other terms in our approximation of $\bar{w}(t)$:

\begin{align}
\bar{w}(t) \approx c_1(0)exp(\frac{\lambda_1 t}{\tau})\bar{e}_1
\end{align}
where $\lambda_1$ is the largest eigenvalue.

So we find that the weight vector is proportional to the first eigenvector of the covariance matrix of the inputs:

\begin{align}
\bar{w}(t) \propto \bar{e}_1
\end{align}

By definition, the first eigenvector of the covariance matrix of the inputs is the first principal component!

In [ ]:
t_vec = np.arange(0, 1000)
plt.plot(np.exp(.6*t_vec), 'r')
plt.plot(np.exp(.9*t_vec), 'g')

## Section 2.3: Interpretation of Hebbian Learning & PCA

So the weight vector learned is the first principal component of the input data. We proved it for the covariance-based rule, but this is also true for the other two learning rules (although those assume that each input neuron has mean 0 firing rate so allows negative firing rates). This is the optimal choice for the weight vector if each input data point is represented by and reconstructed by a single number.

Remember that we project onto the first principal component by taking the dot product of that component and the input vector. This is what the output neuron is doing if the weight vector is the first principal component! 

Remember from linear algebra that the length of the projection of the input vector onto the weight vector is given by:  $||proj_{\bar{w}}\bar{u}|| = \frac{\bar{w}\cdot\bar{u}}{||\bar{w}||}$. 



So the output neuron is computing the length of the projection of the inputs onto the weight vector times the length of the weight vector. So, it's computing something proportional to the length of the projection of the inputs onto the first principal component. Essentially, the output firing rate tells you how far along that weight vector/first principal component the data point is. This is the best possible representation of the input data using only one number per data point - it explains the maximal variance in the input data and leads to the best reconstructions of the data. So the neuron is representing the input data as efficiently and well as possible by being a principal component analyzer!

Let's go back to our simulation from Section 2.1 and look at the output firing rate for a couple inputs. The inputs are now color coded and we show the projection of each input onto the weight vector. The output firing rate will equal the distance from the origin to that projected point, times the length of the weight vector. 

In [ ]:
fig, axes = plt.subplots(1, 3, figsize = (15, 5))

colors = np.arange(10)

axes[0].scatter(u[:10, 0], u[:10, 1], c = colors, cmap = 'jet')
axes[0].plot([0, w[0]], [0, w[1]], '-r', linewidth = 4)
axes[0].plot(w[0], w[1], '-or', linewidth = 4, label = 'Learned weights')
axes[0].set(xlabel = r'$u_1$', ylabel = r'$u_2$', title = 'Inputs & weight vector')
axes[0].legend()

projected_lengths = u[:10] @ w / np.linalg.norm(w) 
projected_points = projected_lengths[:, None] * (w[None, :] / np.linalg.norm(w))
axes[1].scatter(projected_points[:10, 0], projected_points[:10, 1], c = colors, cmap = 'jet')
axes[1].plot([0, w[0]], [0, w[1]], '-r', linewidth = 4)
axes[1].plot(w[0], w[1], '-or', linewidth = 4, label = 'Learned weights')
axes[1].set(xlabel = r'$u_1$', ylabel = r'$u_2$', title = 'Projected inputs')
axes[1].legend()

output_firing_rates = u[:10] @ w
axes[2].scatter(np.zeros((10, )), output_firing_rates, c = colors, cmap = 'jet')
axes[2].set( ylabel = r'$v$', title = 'Output firing rate')


Since we know the output firing rate and the weight vector, we could reconstruct the data point to its place in the middle column from just the output firing rate - which is as good a reconstruction as we can get in this context.

# Section 3: Grid cell example

TBD
